# **Pneumonia detection using x-ray images**



In [7]:
from google.colab import drive
drive.mount('/content/Gdrive')

Mounted at /content/Gdrive


In [8]:
import os
%cd /content/Gdrive/MyDrive/project/pnemonia_detection
HOME = os.getcwd()


/content/Gdrive/MyDrive/project/pnemonia_detection


In [ ]:
!pip install gradio langchain
!pip install -U langchain-openai

In [ ]:
!pip install --upgrade tensorflow

# Pneumonia  classifer

In [20]:
import os
import itertools

# import data handling tools
import numpy as np
import pandas as pd

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras

# ignore the warnings
import warnings
warnings.filterwarnings('ignore')

In [12]:
import cv2

def Load_classifier ():
  # Load the model
  model = tf.keras.models.load_model('best_model.h5')
  return model

def annotator(img, class_label):
  pt1 = (50, 45)
  text = class_label
  text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
  text_w, text_h = text_size[0]
 # Draw the rectangle
  cv2.line(img, pt1, (text_w+50, 45), (255,255,255), 25)
  cv2.putText(img, text, (50, 50),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0,0,0), 2)

def process_data(img):
    # img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img/255.0
    img = np.reshape(img, (224,224,1))

    return img

def run_inference(img):
  img0 = cv2.resize(img, (512, 512))
  global class_label
  classes = ['Healthy', 'Pneumonia']
  model = Load_classifier()
  p_img = process_data(img)
  p_img = np.expand_dims(p_img, axis=0)


  output = model.predict(p_img)
  # print(output)
  max_index = np.argmax(output)
  class_label = classes[max_index]
  annotator(img0, class_label)

  return img0




#chatbot

In [19]:
os.environ["OPENAI_API_KEY"] ='sk-TEbRNi8lPk9t03I99yGIT3BlbkFJ9pJW1ZbgKFSCfWhxy'    # paste your own key

# Chatbot function

In [21]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

model_name = 'gpt-3.5-turbo'

llm = ChatOpenAI(model_name=model_name, temperature=0.3)

def getResponse (user_message):

  pneumonia_info = """Pneumonia is a common respiratory infection that inflames the air sacs in one or both lungs.
                       It can be caused by bacteria, viruses, or fungi, with bacterial and viral pneumonia being
                       the most prevalent types.Symptoms of pneumonia often include cough, fever, difficulty breathing,
                       chest pain, fatigue, and sometimes nausea and vomiting. The severity of symptoms can vary depending
                        on factors such as age, overall health, and the specific cause of pneumonia.

                      Treatment for pneumonia typically involves antibiotics for bacterial pneumonia, antiviral medications
                      for viral pneumonia, and supportive care such as rest, staying hydrated, and managing symptoms like fever and cough.
                       In severe cases, hospitalization may be necessary, especially for those with weakened immune systems,
                       the elderly, or young children. Oxygen therapy and intravenous fluids may be administered in hospitalized
                       patients to support recovery. Early detection and prompt treatment are crucial in managing pneumonia
                       effectively and preventing complications."""

  context_info = pneumonia_info

  conversation_template  = f"""
                you are a pneumonia medical expert and assistant conversational assistant that help doctors and patients in pneumonia classification informations.
                you are provided with some information about the pneumonia diagnosed.

                The doctor or patient will ask some questions based on this and you must use your own knowledge to answer those questions.

              Information on the pneumonia
              Context: {context_info}

              user message: {user_message}
            """

  #initialize the embedchain bot
  PROMPT = PromptTemplate(
        template=conversation_template,
        input_variables= ['context_info','user_message']
    )

  chain = LLMChain(llm=llm, prompt=PROMPT)

  response = chain.predict(context_info = context_info, user_message = user_message)

  return response



#Application interface

In [15]:
import gradio as gr
import numpy as np
from PIL import Image, ImageDraw

In [16]:

# Create Gradio interface
image_input = gr.Image(label="Upload Image", height = 500, width= 700)
# print(image_input)
object_detection_output = gr.Image(label="Pneumonia Detection", height = 500, width= 700)
chat_input = gr.Textbox(lines=2, label="Ask a question")
chat_output = gr.Textbox(label="Chatbot Response")

title = "Pneumonia Detection"
title1 = "Chatbot"
description = "Upload a X--ray image, and ask questions about the result."


#Lunch App

In [17]:
import time, random

with gr.Blocks() as demo1:
    with gr.Row():
        with gr.Column(scale=1):
            detection_interface = gr.Interface(
            fn= run_inference,
            inputs=image_input,
            outputs=object_detection_output,
            title=title,
            description=description,
            allow_flagging='never'  # Disable flagging for simplicity
        )

    with gr.Row():
        with gr.Column(scale=1):
              chat_interface = gr.Interface(
              fn=getResponse,
              inputs=chat_input,
              outputs=chat_output,
              title=title1,
              description="Ask questions about the detection.",
              allow_flagging='never'  # Disable flagging for simplicity
          )

In [18]:
demo1.launch(share=True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://154df7bd870e992b79.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://154df7bd870e992b79.gradio.live
